# Meteorological data

Description: https://en.ilmatieteenlaitos.fi/open-data

Example to get weather data from certain observation point (fmisid=100971) which is in Kaisaniemi, in the heart of Helsinki.
https://opendata.fmi.fi/wfs?request=getFeature&storedquery_id=fmi::observations::weather::timevaluepair&fmisid=100971&starttime=2019-09-05T09:00:00Z&endtime=2019-09-05T11:00:00Z

Observation stations: https://en.ilmatieteenlaitos.fi/observation-stations
Capabilities: https://opendata.fmi.fi/wfs?request=getCapabilities
Description of observation parameters: https://opendata.fmi.fi/meta?observableProperty=observation&language=eng


In [1]:
import urllib.request, gzip, pytz
import pandas as pd
import numpy as np
import xml.etree.ElementTree as et
from datetime import datetime, timedelta

datalink = 'https://opendata.fmi.fi/'

dateformat = '%Y-%m-%dT%H:%M:%SZ'
utcTimezone = pytz.timezone('UTC')
localTimezone = pytz.timezone('Europe/Helsinki')

Get data about weather in kaisaniemi for one day, 60 min intervals

In [2]:
date='2019-09-05'
req = urllib.request.Request(datalink+'wfs?request=getFeature&storedquery_id=fmi::observations::weather::timevaluepair&fmisid=100971&starttime='+date+'T00:00:00Z&endtime='+date+'T24:00:00Z&timestep=60')
req.add_header('Accept-encoding', 'gzip')
res = urllib.request.urlopen(req)
data= gzip.decompress(res.read())
dataroot = et.fromstring(data)

Get some description of data.
- t2m: temperature
- ws_10min: wind speed
- r_1h: Precipitation amount
- snow_aws: Snow depth

In [3]:
dti = pd.date_range(date, periods=24, freq="H")
df = pd.DataFrame(index=dti)
for m in dataroot:
    for x in m[0]:
        if x.tag=='{http://www.opengis.net/om/2.0}result':
            ptype=x[0].attrib['{http://www.opengis.net/gml/3.2}id'].replace('obs-obs-1-1-','')
            series = pd.Series(dtype=np.float64, name=ptype)
            for y in x[0]: 
                for z in y[0]:
                    if z.tag=='{http://www.opengis.net/waterml/2.0}time':
                        time=datetime.strptime(z.text, dateformat)
                    if z.tag=='{http://www.opengis.net/waterml/2.0}value':
                        value=float(z.text)
                series[time]=value
            df=pd.concat([df,series],axis=1)
df=df.rename(columns={'t2m':'Temperature', 'ws_10min':'Wind speed', 'r_1h':'Precipitation amount', 'snow_aws':'Snow depth'})
df=df.drop(['wg_10min','wd_10min','ri_10min','rh','td','vis','n_man','p_sea','wawa'], axis=1)
df
          

,Temperature,Wind speed,Precipitation amount,Snow depth
2019-09-05 00:00:00,13.6,1.2,0.0,-1.0
2019-09-05 01:00:00,15.6,1.8,0.0,-1.0
2019-09-05 02:00:00,15.3,3.3,0.0,-1.0
2019-09-05 03:00:00,15.2,4.6,0.0,-1.0
2019-09-05 04:00:00,15.3,4.7,0.0,-1.0
2019-09-05 05:00:00,15.8,5.3,0.0,-1.0
2019-09-05 06:00:00,15.9,4.6,0.0,-1.0
2019-09-05 07:00:00,16.2,5.0,0.0,-1.0
2019-09-05 08:00:00,16.6,5.8,0.0,-1.0
2019-09-05 09:00:00,17.5,5.4,0.0,-1.0


Load measurement descriptions

In [4]:
req = urllib.request.Request(datalink+'meta?observableProperty=observation&language=eng')
req.add_header('Accept-encoding', 'gzip')
res = urllib.request.urlopen(req)
descriptions = et.fromstring(gzip.decompress(res.read()))